# 1. Import libraries

In [1]:
#----------------------------Reproducible----------------------------------------------------------------------------------------
import numpy as np
import random as rn
import os

seed=0
os.environ['PYTHONHASHSEED'] = str(seed)

np.random.seed(seed)
rn.seed(seed)

#----------------------------Reproducible----------------------------------------------------------------------------------------

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

#--------------------------------------------------------------------------------------------------------------------------------
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cm
%matplotlib inline
matplotlib.style.use('ggplot')

import random
import scipy.sparse as sparse
import scipy.io

from keras.utils import to_categorical
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from skfeature.utility import construct_W
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from skfeature.function.sparse_learning_based import MCFS
from skfeature.utility import construct_W
from skfeature.utility.sparse_learning import feature_ranking
import time
from sklearn.linear_model import LinearRegression
from sklearn.impute import SimpleImputer 
import pandas as pd
from skimage import io
from PIL import Image
from sklearn.model_selection import train_test_split
import scipy.sparse as sparse
from keras.datasets import fashion_mnist
import scipy.io

#--------------------------------------------------------------------------------------------------------------------------------
#Import ourslef defined methods
import sys
sys.path.append(r"../Defined")
import Functions as F

Using TensorFlow backend.


# 2. Loading data

In [2]:
data_path="../Dataset/SMK_CAN_187.mat"
Data = scipy.io.loadmat(data_path)

data_arr=Data['X']
label_arr=Data['Y'][:, 0]
label_arr_onehot=label_arr

In [3]:
key_feture_number=64

In [4]:
num_cluster=len(np.unique(label_arr_onehot))

# 3 .Calculation

In [5]:
#--------------------------------------------------------------------------------------------------------------------------------
def write_to_csv(p_data,p_path):
    dataframe = pd.DataFrame(p_data)
    dataframe.to_csv(p_path, mode='a',header=False,index=False,sep=',')
    del dataframe

#--------------------------------------------------------------------------------------------------------------------------------       
def mse_check(train, test):
    LR = LinearRegression(n_jobs = -1)
    LR.fit(train[0], train[1])
    MSELR = ((LR.predict(test[0]) - test[1]) ** 2).mean()
    return MSELR
 
#--------------------------------------------------------------------------------------------------------------------------------       
def cal(p_data_arr,\
        p_label_arr_onehot,\
        p_key_feture_number,\
        p_seed):
    
    C_train_x,C_test_x,C_train_y,C_test_y= train_test_split(p_data_arr,p_label_arr_onehot,test_size=0.2,random_state=p_seed)

    os.environ['PYTHONHASHSEED'] = str(p_seed)
    np.random.seed(p_seed)
    rn.seed(p_seed)
    
    train_feature=C_train_x
    train_label=C_train_y
    test_feature=C_test_x
    test_label=C_test_y
    
    #--------------------------------------------------------------------------------------------------------------------------------
    # construct affinity matrix
    kwargs_W =  {"metric": "euclidean", "neighborMode": "knn", "weightMode": "heatKernel", "k": 5, 't': 1}
    
    t_start = time.time()

    train_W = construct_W.construct_W(train_feature, **kwargs_W)

    # Obtain the scores of features, and sort the feature scores in an ascending order according to the feature scores
    tain_score = MCFS.mcfs(train_feature, n_selected_features=key_feture_number, W=train_W,n_clusters=num_cluster)
    train_idx = MCFS.feature_ranking(tain_score)
    
    t_used=time.time() - t_start
    write_to_csv(np.array([t_used]),"./log/MCFS_time.csv")

    C_train_selected_x = train_feature[:, train_idx[0:key_feture_number]]

    test_W = construct_W.construct_W(test_feature, **kwargs_W)

    # Obtain the scores of features, and sort the feature scores in an ascending order according to the feature scores
    test_score = MCFS.mcfs(test_feature, n_selected_features=key_feture_number, W=test_W,n_clusters=num_cluster)
    test_idx = MCFS.feature_ranking(test_score)

    # Obtain the dataset on the selected features
    C_test_selected_x = test_feature[:, test_idx[0:key_feture_number]]

    # Classification on original features
    train_feature=C_train_x
    train_label=C_train_y
    test_feature=C_test_x
    test_label=C_test_y
    
    orig_train_acc,orig_test_acc=F.ETree(train_feature,train_label,test_feature,test_label,0)
    
    # Classification on selected features
    
    train_feature=C_train_selected_x
    train_label=C_train_y
    test_feature=C_test_selected_x
    test_label=C_test_y

    selec_train_acc,selec_test_acc=F.ETree(train_feature,train_label,test_feature,test_label,0)

    # Linear reconstruction
    train_feature_tuple=(C_train_selected_x,C_train_x)
    test_feature_tuple=(C_test_selected_x,C_test_x)

    reconstruction_loss=mse_check(train_feature_tuple, test_feature_tuple)
    results=np.array([orig_train_acc,orig_test_acc,selec_train_acc,selec_test_acc,reconstruction_loss])
    
    write_to_csv(results.reshape(1,len(results)),"./log/MCFS_results.csv")
    
    return orig_train_acc,orig_test_acc,selec_train_acc,selec_test_acc,reconstruction_loss

In [6]:
p_data_arr=data_arr
p_label_arr_onehot=label_arr_onehot
p_key_feture_number=key_feture_number

In [ ]:
for p_seed in np.arange(0,50):
    orig_train_acc,orig_test_acc,selec_train_acc,selec_test_acc,reconstruction_loss=cal(p_data_arr,\
                                                                                        p_label_arr_onehot,\
                                                                                        p_key_feture_number,\
                                                                                        p_seed)

/usr/local/lib/python3.7/site-packages/sklearn/linear_model/_least_angle.py:577: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 50 iterations, i.e. alpha=2.447e-03, with an active set of 38 regressors, and the smallest cholesky pivot element being 2.980e-08. Reduce max_iter or increase eps parameters.
  ConvergenceWarning)
/usr/local/lib/python3.7/site-packages/sklearn/linear_model/_least_angle.py:577: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 50 iterations, i.e. alpha=2.438e-03, with an active set of 38 regressors, and the smallest cholesky pivot element being 6.144e-08. Reduce max_iter or increase eps parameters.
  ConvergenceWarning)
/usr/local/lib/python3.7/site-packages/sklearn/linear_model/_least_angle.py:577: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 50 iterations, i.e. alpha=2.432e-03, with an active set of 38 regressors, and the smallest cholesky pivot el

Training accuracy： 1.0
Training accuracy： 1.0
Testing accuracy： 0.7368421052631579
Testing accuracy： 0.7368421052631579
Training accuracy： 1.0
Training accuracy： 1.0
Testing accuracy： 0.4473684210526316
Testing accuracy： 0.4473684210526316


/usr/local/lib/python3.7/site-packages/sklearn/linear_model/_least_angle.py:577: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 49 iterations, i.e. alpha=4.509e-03, with an active set of 38 regressors, and the smallest cholesky pivot element being 5.162e-08. Reduce max_iter or increase eps parameters.
  ConvergenceWarning)
/usr/local/lib/python3.7/site-packages/sklearn/linear_model/_least_angle.py:577: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 49 iterations, i.e. alpha=4.365e-03, with an active set of 38 regressors, and the smallest cholesky pivot element being 6.829e-08. Reduce max_iter or increase eps parameters.
  ConvergenceWarning)
/usr/local/lib/python3.7/site-packages/sklearn/linear_model/_least_angle.py:577: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 49 iterations, i.e. alpha=4.318e-03, with an active set of 38 regressors, and the smallest cholesky pivot el

Training accuracy： 1.0
Training accuracy： 1.0
Testing accuracy： 0.631578947368421
Testing accuracy： 0.631578947368421
Training accuracy： 1.0
Training accuracy： 1.0
Testing accuracy： 0.5526315789473685
Testing accuracy： 0.5526315789473685
Training accuracy： 1.0
Training accuracy： 1.0
Testing accuracy： 0.6578947368421053
Testing accuracy： 0.6578947368421053
Training accuracy： 1.0
Training accuracy： 1.0
Testing accuracy： 0.47368421052631576
Testing accuracy： 0.47368421052631576


/usr/local/lib/python3.7/site-packages/sklearn/linear_model/_least_angle.py:577: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 49 iterations, i.e. alpha=2.668e-03, with an active set of 37 regressors, and the smallest cholesky pivot element being 3.161e-08. Reduce max_iter or increase eps parameters.
  ConvergenceWarning)
/usr/local/lib/python3.7/site-packages/sklearn/linear_model/_least_angle.py:577: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 53 iterations, i.e. alpha=2.674e-03, with an active set of 39 regressors, and the smallest cholesky pivot element being 5.674e-08. Reduce max_iter or increase eps parameters.
  ConvergenceWarning)


...

Training accuracy： 1.0
Training accuracy： 1.0
Testing accuracy： 0.7368421052631579
Testing accuracy： 0.7368421052631579
Training accuracy： 1.0
Training accuracy： 1.0
Testing accuracy： 0.5526315789473685
Testing accuracy： 0.5526315789473685
Training accuracy： 1.0
Training accuracy： 1.0
Testing accuracy： 0.631578947368421
Testing accuracy： 0.631578947368421
Training accuracy： 1.0
Training accuracy： 1.0
Testing accuracy： 0.5
Testing accuracy： 0.5
Training accuracy： 1.0
Training accuracy： 1.0
Testing accuracy： 0.7105263157894737
Testing accuracy： 0.7105263157894737
Training accuracy： 1.0
Training accuracy： 1.0
Testing accuracy： 0.47368421052631576
Testing accuracy： 0.47368421052631576
Training accuracy： 1.0
Training accuracy： 1.0
Testing accuracy： 0.6842105263157895
Testing accuracy： 0.6842105263157895
Training accuracy： 1.0
Training accuracy： 1.0
Testing accuracy： 0.5
Testing accuracy： 0.5


/usr/local/lib/python3.7/site-packages/sklearn/linear_model/_least_angle.py:577: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 50 iterations, i.e. alpha=4.299e-03, with an active set of 38 regressors, and the smallest cholesky pivot element being 2.356e-08. Reduce max_iter or increase eps parameters.
  ConvergenceWarning)
/usr/local/lib/python3.7/site-packages/sklearn/linear_model/_least_angle.py:577: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 50 iterations, i.e. alpha=4.244e-03, with an active set of 38 regressors, and the smallest cholesky pivot element being 1.490e-08. Reduce max_iter or increase eps parameters.
  ConvergenceWarning)
/usr/local/lib/python3.7/site-packages/sklearn/linear_model/_least_angle.py:577: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 50 iterations, i.e. alpha=4.214e-03, with an active set of 38 regressors, and the smallest cholesky pivot el

Training accuracy： 1.0
Training accuracy： 1.0
Testing accuracy： 0.7368421052631579
Testing accuracy： 0.7368421052631579
Training accuracy： 1.0
Training accuracy： 1.0
Testing accuracy： 0.6052631578947368
Testing accuracy： 0.6052631578947368
Training accuracy： 1.0
Training accuracy： 1.0
Testing accuracy： 0.6842105263157895
Testing accuracy： 0.6842105263157895
Training accuracy： 1.0
Training accuracy： 1.0
Testing accuracy： 0.5789473684210527
Testing accuracy： 0.5789473684210527


/usr/local/lib/python3.7/site-packages/sklearn/linear_model/_least_angle.py:577: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 49 iterations, i.e. alpha=4.939e-03, with an active set of 38 regressors, and the smallest cholesky pivot element being 9.424e-08. Reduce max_iter or increase eps parameters.
  ConvergenceWarning)
/usr/local/lib/python3.7/site-packages/sklearn/linear_model/_least_angle.py:577: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 49 iterations, i.e. alpha=4.911e-03, with an active set of 38 regressors, and the smallest cholesky pivot element being 8.495e-08. Reduce max_iter or increase eps parameters.
  ConvergenceWarning)
/usr/local/lib/python3.7/site-packages/sklearn/linear_model/_least_angle.py:577: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 49 iterations, i.e. alpha=4.865e-03, with an active set of 38 regressors, and the smallest cholesky pivot el

Training accuracy： 1.0
Training accuracy： 1.0
Testing accuracy： 0.7894736842105263
Testing accuracy： 0.7894736842105263
Training accuracy： 1.0
Training accuracy： 1.0
Testing accuracy： 0.5
Testing accuracy： 0.5
